In [1]:
def download_Project():
  !pip install gdown
  !gdown https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL
  !unzip str(pathlib.Path("Project_data.zip").resolve())

download_Project()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 KB 3.4 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Downloading...
From (original): https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL
From (redirected): https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL&confirm=t&uuid=16c5f201-07fd-480d-88ec-8c80cb8b2313
To: /home/Gesture_Recognition/Project_data.zip
100%|██████████████████████████████████████| 1.71G/1.71G [02:01<00:00, 14.1MB/s]
Archive:  /home/Gesture_Recognition/Project_data.zip
   creating: Project_data/
   creating: Project_data/train/
  inflating: Project_data/train.csv  
   creating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00012.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00014.png  
  inf

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd

import random as rn
import numpy.random as nrn
import tensorflow as tf
import os
import pathlib

from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Dropout, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.applications import ResNet50, ResNet101

import PIL
from PIL import Image
import cv2

import matplotlib.pyplot as plt

In [2]:
batch_size = 10
num_epochs = 50
image_shape = (224, 224, 3)
vid_shape = (15, 224, 224, 3)

rn.seed(37)
nrn.seed(37)
tf.random.set_seed(37)


def create_dfs(data_folder):
    train_df = pd.read_csv(f"{data_folder}/train.csv", sep=";", header=None)
    train_df.columns = ['folder', 'type', 'class']
    val_df = pd.read_csv(f"{data_folder}/val.csv", sep=";", header=None)
    val_df.columns = ['folder', 'type', 'class']
    return train_df, val_df


def get_folder_list(df_csv):
  fptr = open(df_csv)
  folder_paths = fptr.readlines()
  fptr.close()
  return folder_paths


def get_batch_details(folder_list, batch_size):
  num_batches = len(folder_list) // batch_size
  folders_remain = len(folder_list) - (num_batches * batch_size)
  return num_batches, folders_remain


def create_batch_folders(batch_size, img_ids, image_shape):
         ## batch_images                                                                               ## batch_labels
  return (np.zeros(shape = (batch_size, img_ids, image_shape[0], image_shape[1], 3)), np.zeros(shape = (batch_size, 5)))


def process_images(imgs_path, batch_images, folder, image_shape, img_ids, img_crop):
    ## get images and labels for each folder in the batch
    imgs = os.listdir(imgs_path)
    for idx, ind in enumerate(img_ids):
        ## read image
        image = cv2.imread(f'{imgs_path}/{imgs[ind]}').astype(np.float32)
        ## resize image
        image = cv2.resize(image, (image_shape[0], image_shape[1]), interpolation = cv2.INTER_LINEAR)
        ## Seperating them into their respective R, G, B channels and normalizing the values
        ## Load labels for each image
        batch_images[folder, idx, :, :, 0] = image[:,:,0]/255.0   ## R
        batch_images[folder, idx, :, :, 1] = image[:,:,1]/255.0   ## G
        batch_images[folder, idx, :, :, 2] = image[:,:,2]/255.0   ## B


## path => folder path as string
## folder_list => list of folders in train/test folder as list
## batch_size => chosen batch size as int
## image_shape => as tuple (image_height, image_width)
def run_generator(path, folder_list, batch_size, image_shape, img_crop):
  num_batches, folders_remain = get_batch_details(folder_list, batch_size)
  ## set how many images to view from total images of the video
  img_ids = list(range(0,30,2))
  while True:
    folder_list = np.random.permutation(folder_list)
    for batch in range(num_batches):
      batch_images, batch_labels = create_batch_folders(batch_size, len(img_ids), image_shape)
      ## get folders for that batch
      for folder in range(batch_size):
        ## get images and labels for each folder in the batch
        process_images(f'{path}/{folder_list[folder + (batch * batch_size)].split(";")[0]}', batch_images, folder, image_shape, img_ids, 0)
          ## Load labels for each image
        batch_labels[folder, int(folder_list[folder + (batch * batch_size)].strip().split(';')[2])] = 1
      yield batch_images, batch_labels

    ## Process the remaining folders
    if folders_remain > 0:
      batch_images, batch_labels = create_batch_folders(folders_remain, len(img_ids), image_shape)  
      folder_rem = folder_list[-folders_remain:]
      for folder in range(folders_remain):
        process_images(f'{path}/{folder_rem[folder].split(";")[0]}', batch_images, folder, image_shape, img_ids, 0)
        batch_labels[folder, int(folder_rem[folder].strip().split(';')[2])] = 1
      yield batch_images, batch_labels


def model_log_save():
  curr_dt_time = datetime.now()

  model_name = 'model_cnn3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

  if not os.path.exists(model_name):
      os.mkdir(model_name)

  filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.weights.h5'
  checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=True, mode='auto')
  LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_lr=0.001, cooldown=2)
  callbacks_list = [checkpoint, LR]

  return callbacks_list


def folders_list(data_folder):
    fptr = open(data_folder)
    data_paths = fptr.readlines()
    fptr.close()
    folder_lists = np.random.permutation(data_paths)
    return folder_lists


def total_epochs(train_folder_list, val_folder_list, batch_size):
    if (len(train_folder_list)%batch_size) == 0:
        steps_per_epoch = int(len(train_folder_list)/batch_size)
    else:
        steps_per_epoch = int(len(train_folder_list)//batch_size) + 1

    if (len(val_folder_list)%batch_size) == 0:
        validation_steps = int(len(val_folder_list)/batch_size)
    else:
        validation_steps = int(len(val_folder_list)//batch_size) + 1

    return steps_per_epoch, validation_steps


def run_model(gru_opt, process_type):
    data_folder = str(pathlib.Path("Project_data").resolve())

    train_df, val_df = create_dfs(data_folder)
    folder_lists = folders_list(f'{data_folder}/train.csv')
    split_val = len(folder_lists) - (len(folder_lists)*(20)//100)
    train_folder_list = folder_lists[:split_val]
    val_folder_list = folder_lists[split_val:]

    num_classes = train_df['class'].unique().size

    steps_per_epoch, validation_steps = total_epochs(train_folder_list, val_folder_list, batch_size)

    train_generator = run_generator(f'{data_folder}/train', train_folder_list, batch_size, image_shape, 0)
    val_generator = run_generator(f'{data_folder}/train', val_folder_list, batch_size, image_shape, 0)

    if gru_opt:
        model = gru_model(num_classes)
        model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    else:
        model = cnn3D_model(vid_shape, num_classes)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    return model, train_generator, val_generator, steps_per_epoch, validation_steps, num_classes


def run_test_model():
    data_folder = str(pathlib.Path("Project_data").resolve())
    folder_lists = folders_list(f'{data_folder}/val.csv')
    testing_generator = test_generator(f'{data_folder}/val', folder_lists, batch_size, image_shape, 0)
    return testing_generator


In [3]:
## 64 + 64 + norm + 128 + 128 + dropout + norm + 256 + 256 + 1024_dense(2) + dropout
def cnn3D_model(vid_shape, num_classes):
    model = Sequential()

    model.add(layers.Input(shape=vid_shape)),

    model.add(layers.Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')),
    model.add(layers.Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')),
    model.add(layers.BatchNormalization()),
    model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),

    model.add(layers.Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu')),
    model.add(layers.Conv3D(filters=128, kernel_size=(3, 3, 3), padding = 'same', activation='relu')),
    model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),

    model.add(layers.Conv3D(filters=256, kernel_size=(3, 3, 3), padding = 'same', activation='relu')),
    model.add(layers.BatchNormalization()),
    model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),
    model.add(layers.Conv3D(filters=512, kernel_size=(3, 3, 3), padding = 'same', activation='relu')),

    ## Flatten and Dense layers
    model.add(layers.GlobalAveragePooling3D()),
    model.add(layers.Dense(1024, activation='relu')),
    model.add(layers.Dropout(rate=0.3)),
    model.add(layers.Dense(1024, activation='relu')),
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [5]:
## 64 + 64 + norm + 128 + 128 + 256 + dropout(0.3) + norm + 512 + 1024_dense(2) + dropout(0.3)
cnn3D_model, train_generator, val_generator, steps_per_epoch, validation_steps, num_classes = run_model(0, 3)
cnn3D_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 13, 222, 222, 64)  5248      
                                                                 
 conv3d_1 (Conv3D)           (None, 11, 220, 220, 64)  110656    
                                                                 
 batch_normalization (BatchN  (None, 11, 220, 220, 64)  256      
 ormalization)                                                   
                                                                 
 max_pooling3d (MaxPooling3D  (None, 11, 110, 110, 64)  0        
 )                                                               
                                                                 
 conv3d_2 (Conv3D)           (None, 9, 108, 108, 128)  221312    
                                                                 
 conv3d_3 (Conv3D)           (None, 9, 108, 108, 128)  4

2025-01-08 11:45:57.962842: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2025-01-08 11:45:57.962896: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14800 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:1b:00.0, compute capability: 7.5


In [5]:
## 64 + 64 + norm + 128 + 128 + 256 + dropout(0.3) + norm + 512 + 1024_dense(2) + dropout(0.3)
cnn3D_model.fit(
                    train_generator, 
                    steps_per_epoch    = steps_per_epoch,
                    epochs             = num_epochs,
                    verbose            = 1,
                    callbacks          = model_log_save(),
                    validation_data    = val_generator,
                    validation_steps   = validation_steps,
                    class_weight       = None,
                    initial_epoch      = 0
                 )

Epoch 1/50


2025-01-08 09:46:19.451900: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


54/54 [==============================] - ETA: 0s - loss: 21.3840 - categorical_accuracy: 0.2486
Epoch 00001: saving model to model_cnn3D_2025-01-0809_46_17.088253/model-00001-21.38396-0.24859-66.01678-0.22727.weights.h5
54/54 [==============================] - 177s 3s/step - loss: 21.3840 - categorical_accuracy: 0.2486 - val_loss: 66.0168 - val_categorical_accuracy: 0.2273 - lr: 0.0100
Epoch 2/50
54/54 [==============================] - ETA: 0s - loss: 1.5881 - categorical_accuracy: 0.2467
Epoch 00002: saving model to model_cnn3D_2025-01-0809_46_17.088253/model-00002-1.58806-0.24670-5.81804-0.24242.weights.h5
54/54 [==============================] - 114s 2s/step - loss: 1.5881 - categorical_accuracy: 0.2467 - val_loss: 5.8180 - val_categorical_accuracy: 0.2424 - lr: 0.0100
Epoch 3/50
54/54 [==============================] - ETA: 0s - loss: 1.5549 - categorical_accuracy: 0.2599
Epoch 00003: saving model to model_cnn3D_2025-01-0809_46_17.088253/model-00003-1.55489-0.25989-2.16487-0.20455

In [19]:
## Load selected model
file_path = str(pathlib.Path("model_cnn3D_2025-01-0809_46_17.088253/model-00050-0.41419-0.82674-0.55758-0.79545.weights.h5").resolve())
vid_shape = (15, 224, 224, 3)
test_model = cnn3D_model(vid_shape, 5)
test_model.load_weights(file_path)
test_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
test_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_18 (Conv3D)          (None, 13, 222, 222, 64)  5248      
                                                                 
 conv3d_19 (Conv3D)          (None, 11, 220, 220, 64)  110656    
                                                                 
 batch_normalization_6 (Batc  (None, 11, 220, 220, 64)  256      
 hNormalization)                                                 
                                                                 
 max_pooling3d_9 (MaxPooling  (None, 11, 110, 110, 64)  0        
 3D)                                                             
                                                                 
 conv3d_20 (Conv3D)          (None, 9, 108, 108, 128)  221312    
                                                                 
 conv3d_21 (Conv3D)          (None, 9, 108, 108, 128) 

In [10]:
def test_generator(path, folder_list, batch_size, image_shape, img_crop):
    num_batches, folders_remain = get_batch_details(folder_list, batch_size)
    ## set how many images to view from total images of the video
    img_ids = list(range(0,30,2))
    folder_list = np.random.permutation(folder_list)
    for batch in range(num_batches):
      batch_images, batch_labels = create_batch_folders(batch_size, len(img_ids), image_shape)
      ## get folders for that batch
      for folder in range(batch_size):
        ## get images and labels for each folder in the batch
        process_images(f'{path}/{folder_list[folder + (batch * batch_size)].split(";")[0]}', batch_images, folder, image_shape, img_ids, 0)
          ## Load labels for each image
        batch_labels[folder, int(folder_list[folder + (batch * batch_size)].strip().split(';')[2])] = 1
      yield batch_images, batch_labels

    ## Process the remaining folders
    if folders_remain > 0:
      batch_images, batch_labels = create_batch_folders(folders_remain, len(img_ids), image_shape)  
      folder_rem = folder_list[-folders_remain:]
      for folder in range(folders_remain):
        process_images(f'{path}/{folder_rem[folder].split(";")[0]}', batch_images, folder, image_shape, img_ids, 0)
        batch_labels[folder, int(folder_rem[folder].strip().split(';')[2])] = 1
      yield batch_images, batch_labels


In [14]:
testing_generator = run_test_model()
test_model.evaluate(testing_generator)

10/10 [==============================] - 7s 640ms/step - loss: 0.6466 - categorical_accuracy: 0.7500


[0.6465699672698975, 0.75]

In [ ]:
###############################################################################################################################################

In [6]:
## Running the model on cropped images

In [5]:
## 64 + 64 + norm + 128 + 128 + 256 + dropout(0.3) + norm + 512 + 1024_dense(2) + dropout(0.3)
cnn3D_crop_model, train_generator, val_generator, steps_per_epoch, validation_steps, num_classes = run_model(1, 2)
cnn3D_crop_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 13, 158, 198, 64)  5248      
                                                                 
 conv3d_1 (Conv3D)           (None, 11, 156, 196, 64)  110656    
                                                                 
 batch_normalization (BatchN  (None, 11, 156, 196, 64)  256      
 ormalization)                                                   
                                                                 
 max_pooling3d (MaxPooling3D  (None, 11, 78, 98, 64)   0         
 )                                                               
                                                                 
 conv3d_2 (Conv3D)           (None, 9, 76, 96, 128)    221312    
                                                                 
 conv3d_3 (Conv3D)           (None, 9, 76, 96, 128)    4

2025-01-07 21:12:29.815903: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2025-01-07 21:12:29.815967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14800 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:1c:00.0, compute capability: 7.5


In [6]:
## 64 + 64 + norm + 128 + 128 + 256 + dropout(0.3) + norm + 512 + 1024_dense(2) + dropout(0.3)
cnn3D_crop_model.fit(
                    train_generator, 
                    steps_per_epoch    = steps_per_epoch,
                    epochs             = num_epochs,
                    verbose            = 1,
                    validation_data    = val_generator,
                    validation_steps   = validation_steps,
                    class_weight       = None,
                    initial_epoch      = 0
                 )

Epoch 1/50


2025-01-07 21:12:55.046727: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


43/43 [==============================] - 94s 2s/step - loss: 24.9215 - categorical_accuracy: 0.2133 - val_loss: 959.2944 - val_categorical_accuracy: 0.1400
Epoch 2/50
43/43 [==============================] - 47s 1s/step - loss: 1.6146 - categorical_accuracy: 0.2000 - val_loss: 13.3236 - val_categorical_accuracy: 0.1600
Epoch 3/50
43/43 [==============================] - 39s 919ms/step - loss: 1.5207 - categorical_accuracy: 0.3000 - val_loss: 13.4260 - val_categorical_accuracy: 0.1000
Epoch 4/50
43/43 [==============================] - 40s 921ms/step - loss: 1.4281 - categorical_accuracy: 0.3300 - val_loss: 8.3622 - val_categorical_accuracy: 0.2000
Epoch 5/50
43/43 [==============================] - 39s 918ms/step - loss: 1.5092 - categorical_accuracy: 0.3133 - val_loss: 1.7344 - val_categorical_accuracy: 0.1200
Epoch 6/50
43/43 [==============================] - 39s 902ms/step - loss: 1.3425 - categorical_accuracy: 0.3467 - val_loss: 1.3467 - val_categorical_accuracy: 0.2200
Epoch 7/50

In [ ]:
                                                ########################################
                                                ## Build a model using ResNet50 + GRU ##
                                                ########################################

In [3]:
def gru_model(num_classes):
    gru_model = Sequential()

    resnet_model = ResNet101(
        include_top=False,
        weights='imagenet',
        pooling=None,
        input_shape=(224, 224, 3)
    )

    splitAt = 95
    for layer in resnet_model.layers[:splitAt]:
        layer.trainable = False
    for layer in resnet_model.layers[splitAt:]:
        layer.trainable = True
    # for layer in resnet_model.layers:
    #     layer.trainable = False

    gru_model.add(layers.TimeDistributed(resnet_model, input_shape = (15, 224, 224, 3))),
    gru_model.add(layers.TimeDistributed(layers.GlobalAveragePooling2D())),
    gru_model.add(layers.GRU(units=512, return_sequences=False)),
    gru_model.add(layers.Dropout(rate=0.2)),
    gru_model.add(layers.Dense(num_classes, activation='softmax'))

    return gru_model

In [4]:
gru_model, train_generator, val_generator, steps_per_epoch, validation_steps, num_classes = run_gru_model()
gru_model.summary()

2025-01-07 16:36:25.561644: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2025-01-07 16:36:25.561720: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14800 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:1c:00.0, compute capability: 7.5


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 15, 7, 7, 2048)   42658176  
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 15, 2048)         0         
 tributed)                                                       
                                                                 
 gru (GRU)                   (None, 512)               3935232   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 5)                 2565      
                                                                 
Total params: 46,595,973
Trainable params: 43,262,725
No

In [5]:
gru_model.fit(
                train_generator, 
                steps_per_epoch    = steps_per_epoch,
                epochs             = num_epochs,
                verbose            = 1,
                callbacks          = model_log_save(),
                validation_data    = val_generator,
                validation_steps   = validation_steps,
                class_weight       = None,
                initial_epoch      = 0
             )

Epoch 1/50


2025-01-07 16:36:44.602329: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


76/76 [==============================] - ETA: 0s - loss: 1.9077 - categorical_accuracy: 0.2938
Epoch 00001: saving model to model_cnn3D_2025-01-0716_36_32.105701/model-00001-1.90766-0.29379-1.76808-0.23485.keras


/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


76/76 [==============================] - 183s 2s/step - loss: 1.9077 - categorical_accuracy: 0.2938 - val_loss: 1.7681 - val_categorical_accuracy: 0.2348 - lr: 0.0100
Epoch 2/50
76/76 [==============================] - ETA: 0s - loss: 1.5357 - categorical_accuracy: 0.3672
Epoch 00002: saving model to model_cnn3D_2025-01-0716_36_32.105701/model-00002-1.53566-0.36723-1.79598-0.25000.keras
76/76 [==============================] - 74s 979ms/step - loss: 1.5357 - categorical_accuracy: 0.3672 - val_loss: 1.7960 - val_categorical_accuracy: 0.2500 - lr: 0.0100
Epoch 3/50
76/76 [==============================] - ETA: 0s - loss: 1.3358 - categorical_accuracy: 0.4689
Epoch 00003: saving model to model_cnn3D_2025-01-0716_36_32.105701/model-00003-1.33579-0.46893-1.73425-0.32576.keras
76/76 [==============================] - 74s 975ms/step - loss: 1.3358 - categorical_accuracy: 0.4689 - val_loss: 1.7342 - val_categorical_accuracy: 0.3258 - lr: 0.0100
Epoch 4/50
76/76 [==============================]